<a href="https://colab.research.google.com/github/Georopeza/Georopeza-Optimizacion-Rutas-QLearning/blob/main/IAAA_P3_FernandezMariangel_OropezaGerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports, Ciudades y distancias

In [ ]:
#Importaciones
import numpy as np
import random
import math
import pandas as pd # Importa pandas para una mejor visualización de la matriz
import matplotlib.pyplot as plt # Importa matplotlib para graficar

# Permite al usuario ingresar el número de ciudades
while True:
    num_ciudades = int(input("Ingrese el número de ciudades (mínimo 3, máximo 10): "))
    if 3 <= num_ciudades <= 15: #de 3 a 10
        break
    else:
        print("Número de ciudades inválido. Por favor, ingrese un número entre 3 y 10.")

# Permite al usuario ingresar las coordenadas o generarlas al azar
coords_input = input("¿Desea ingresar las coordenadas manualmente (m) o generarlas al azar (a)? ")
if coords_input.lower() == 'm':
    coordenadas = []
    for i in range(num_ciudades):
        x = float(input(f"Ingrese la coordenada x para la ciudad {i}: "))
        y = float(input(f"Ingrese la coordenada y para la ciudad {i}: "))
        coordenadas.append((x, y))
else:
    # Genera coordenadas al azar como enteros entre 0 y 100
    coordenadas = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(num_ciudades)]

# Muestra las coordenadas
print("\nCoordenadas de las ciudades:")
for i, (x, y) in enumerate(coordenadas):
    print(f"Ciudad {i}: ({x:.2f}, {y:.2f})")


# Implementa la función para calcular la distancia euclidiana
def distancia_euclidiana(ciudad1, ciudad2):
    return math.sqrt((ciudad1[0] - ciudad2[0])**2 + (ciudad1[1] - ciudad2[1])**2)

# Crea una matriz de distancias
matriz_distancias = np.zeros((num_ciudades, num_ciudades))
for i in range(num_ciudades):
    for j in range(num_ciudades):
        matriz_distancias[i, j] = distancia_euclidiana(coordenadas[i], coordenadas[j]) #llama a la funcion distancia eucladiana para rellenar la matriz

# Muestra la matriz de distancias usando un DataFrame de pandas
print("\nMatriz de distancias:")
distancia_df = pd.DataFrame(matriz_distancias, index=[f'Ciudad {i}' for i in range(num_ciudades)], columns=[f'Ciudad {j}' for j in range(num_ciudades)])
display(distancia_df)



Algoritmo

In [ ]:
# --- Parámetros de Q-Learning ---
alpha = 0.4  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.1  # Tasa de exploración
num_episodios = 1000 # Número de episodios de entrenamiento
epsilon_min = 0.05
decay_rate = 0.995

# Inicializa la tabla Q
# El estado se representa como (ciudad_actual, bitmask_ciudades_visitadas)
# El bitmask indica qué ciudades han sido visitadas (1 si visitada, 0 si no)
# El número de estados es num_ciudades * 2^num_ciudades
# Las acciones son las ciudades a las que se puede mover (0 a num_ciudades-1)
q_table = np.zeros((num_ciudades, 2**num_ciudades, num_ciudades))

# --- Funciones Auxiliares ---

# Función para obtener el índice del estado a partir de la ciudad actual y el bitmask
def get_estado_index(ciudad_actual, bitmask_visitadas):
    return (ciudad_actual, bitmask_visitadas)

# Función para obtener las acciones posibles (ciudades no visitadas)
def get_acciones_posibles(bitmask_visitadas):
    acciones_posibles = []
    for ciudad in range(num_ciudades):
        # Si el bit correspondiente es 0, la ciudad no ha sido visitada
        if (bitmask_visitadas >> ciudad) & 1 == 0:
            acciones_posibles.append(ciudad)
    return acciones_posibles

# Función para actualizar el bitmask de ciudades visitadas
def actualizar_bitmask(bitmask_visitadas, ciudad_visitada):
    return bitmask_visitadas | (1 << ciudad_visitada)

# --- Algoritmo de Q-Learning ---

print("\nEntrenando el agente Q-Learning...")

# Lista para almacenar la distancia del camino encontrado al final de cada episodio para graficar la convergencia
episode_distances = []
large_positive_reward = 500.0 # Define una gran recompensa positiva por completar un tour válido

coords_input = input("¿Desea ingresar la ciudad inicial manualmente (m) o generarla al azar (a)?")
if coords_input.lower() == 'm':
        ciudad_inicio = int(input(f"Ingrese la ciudad con la que desea iniciar: "))
else:
    ciudad_inicio = random.randint(0, num_ciudades - 1)
for episodio in range(num_episodios):
    # El agente comienza en una ciudad aleatoria en cada episodio
    ciudad_actual = ciudad_inicio
    # Inicializa el bitmask indicando que solo la ciudad inicial ha sido visitada
    bitmask_visitadas = 1 << ciudad_actual

    recompensa_total_episodio = 0
    pasos = 0

    # El episodio termina cuando todas las ciudades han sido visitadas O si no hay acciones válidas posibles
    while bitmask_visitadas < (2**num_ciudades) - 1:

        estado_index = get_estado_index(ciudad_actual, bitmask_visitadas)
        acciones_posibles = get_acciones_posibles(bitmask_visitadas)

        # Si no hay acciones posibles (todas las ciudades visitadas), romper el bucle
        if not acciones_posibles:
            break

        # Estrategia Epsilon-Greedy, para elegir la accion
        if random.uniform(0, 1) < epsilon:
            # Exploración: elige una acción aleatoria entre las posibles
            proxima_ciudad = random.choice(acciones_posibles)
        else:
            # Explotación: elige la acción con el mayor valor Q entre las posibles
            q_valores_actuales = q_table[estado_index]
            # Filtra los valores Q para solo las acciones posibles
            q_valores_posibles = [q_valores_actuales[accion] for accion in acciones_posibles]
            max_q = max(q_valores_posibles)
            # Maneja el caso de múltiples acciones con el mismo valor Q máximo
            mejores_acciones = [acciones_posibles[i] for i, q_val in enumerate(q_valores_posibles) if q_val == max_q]
            proxima_ciudad = random.choice(mejores_acciones)

        # Realiza la acción
        # La recompensa inmediata es la distancia negativa
        recompensa = -matriz_distancias[ciudad_actual, proxima_ciudad]
        recompensa_total_episodio += recompensa

        # Calcula el próximo estado
        proximo_bitmask_visitadas = actualizar_bitmask(bitmask_visitadas, proxima_ciudad)
        proximo_estado_index = get_estado_index(proxima_ciudad, proximo_bitmask_visitadas)

        # Actualiza la tabla Q
        # Encuentra el máximo valor Q para el próximo estado (considerando solo acciones posibles desde el próximo estado)
        acciones_posibles_proximo = get_acciones_posibles(proximo_bitmask_visitadas)
        if acciones_posibles_proximo:
            max_q_s_prime_a_prime = np.max([q_table[proximo_estado_index][accion] for accion in acciones_posibles_proximo])
        else:
            # Si todas las ciudades han sido visitadas, el próximo estado es el estado final (regreso al inicio)
            # La recompensa por este último paso se maneja fuera de este bucle while
            max_q_s_prime_a_prime = 0 # No hay recompensas futuras desde un estado terminal


        # Actualiza los valores de Q (Ecuación de Bellman)
        q_table[estado_index][proxima_ciudad] = q_table[estado_index][proxima_ciudad] + alpha * (recompensa + gamma * max_q_s_prime_a_prime - q_table[estado_index][proxima_ciudad])

        # Actualiza el estado actual
        ciudad_actual = proxima_ciudad
        bitmask_visitadas = proximo_bitmask_visitadas
        pasos += 1 #contador de cuantas acciones ha tomado

    # Después de visitar todas las ciudades, el agente regresa a la ciudad de inicio para cerrar el ciclo
    # Verifica si todas las ciudades fueron visitadas (bitmask completo) y si el agente no está ya en el inicio
    if bitmask_visitadas == (2**num_ciudades) - 1 and ciudad_actual != ciudad_inicio:
        estado_index = get_estado_index(ciudad_actual, bitmask_visitadas)
        proxima_ciudad = ciudad_inicio # La única acción posible es regresar al inicio

        # Calcula la recompensa por regresar al inicio
        # Esta recompensa incluye la distancia negativa Y la gran recompensa positiva por completar el tour
        recompensa_retorno = -matriz_distancias[ciudad_actual, proxima_ciudad] + large_positive_reward
        recompensa_total_episodio += recompensa_retorno # Agrega la recompensa neta al total

        # Decaimiento de epsilon para que el agente explore menos y explote lo aprendido
        epsilon = max(epsilon * decay_rate, epsilon_min)

        # No hay un próximo estado después de regresar al inicio en un episodio
        max_q_s_prime_a_prime = 0

        # Actualiza la tabla Q para el último paso (regreso al inicio)
        q_table[estado_index][proxima_ciudad] = q_table[estado_index][proxima_ciudad] + alpha * (recompensa_retorno + gamma * max_q_s_prime_a_prime - q_table[estado_index][proxima_ciudad])

        ciudad_actual = proxima_ciudad # El agente ha regresado al inicio
        pasos += 1
    # Si el bucle terminó pero no todas las ciudades fueron visitadas (no debería ocurrir con la lógica actual, pero como medida de seguridad)
    # o si el agente terminó en el inicio sin visitar todas las ciudades (tour inválido)
    # podríamos agregar una penalización aquí si fuera necesario, pero la estructura de recompensa actual penaliza implícitamente los tours incompletos
    # al no proporcionar la gran recompensa positiva.

    # --- Evaluación al final de cada episodio para seguir la convergencia ---
    # Encuentra el mejor camino usando la tabla Q actual (explotación pura)
    # Comienza desde la ciudad inicial de este episodio para la evaluación
    ciudad_inicio_eval = ciudad_inicio
    camino_encontrado_eval = [ciudad_inicio_eval]
    ciudad_actual_eval = ciudad_inicio_eval
    bitmask_visitadas_eval = 1 << ciudad_inicio_eval
    distancia_total_eval = 0

    while bitmask_visitadas_eval < (2**num_ciudades) - 1:
        estado_index_eval = get_estado_index(ciudad_actual_eval, bitmask_visitadas_eval)
        acciones_posibles_eval = get_acciones_posibles(bitmask_visitadas_eval)

        if not acciones_posibles_eval:
             # Si no hay acciones posibles, romper - esto podría indicar un camino incompleto
             distancia_total_eval = float('inf') # Asigna una distancia muy grande para caminos incompletos
             break

        q_valores_actuales_eval = q_table[estado_index_eval]
        q_valores_posibles_eval = [q_valores_actuales_eval[accion] for accion in acciones_posibles_eval]
        # Maneja el caso en el que todos los valores Q posibles son cero o negativos (estado inicial)
        if not q_valores_posibles_eval:
             # Si no hay acciones posibles en absoluto, algo está mal, romper
             distancia_total_eval = float('inf')
             break
        else:
            max_q_eval = max(q_valores_posibles_eval)
            # Si todos los valores Q son negativos o cero, elige una acción aleatoria entre las posibles para intentar escapar
            if max_q_eval <= 0:
                 proxima_ciudad_eval = random.choice(acciones_posibles_eval)
            else:
                 # Elige la acción con el mayor valor Q positivo
                 mejores_acciones_eval = [acciones_posibles_eval[i] for i, q_val in enumerate(q_valores_posibles_eval) if q_val == max_q_eval]
                 proxima_ciudad_eval = random.choice(mejores_acciones_eval)


        distancia_total_eval += matriz_distancias[ciudad_actual_eval, proxima_ciudad_eval]
        camino_encontrado_eval.append(proxima_ciudad_eval)

        ciudad_actual_eval = proxima_ciudad_eval
        bitmask_visitadas_eval = actualizar_bitmask(bitmask_visitadas_eval, proxima_ciudad_eval)

    # Agrega la distancia para regresar al inicio si se completó un tour completo
    if bitmask_visitadas_eval == (2**num_ciudades) - 1 and ciudad_actual_eval != ciudad_inicio_eval:
        distancia_total_eval += matriz_distancias[ciudad_actual_eval, ciudad_inicio_eval]
        camino_encontrado_eval.append(ciudad_inicio_eval)
    elif bitmask_visitadas_eval < (2**num_ciudades) - 1:
        # Si el camino fue incompleto, establece la distancia a infinito o un valor grande
        distancia_total_eval = float('inf')

    # Almacena la distancia para este episodio
    episode_distances.append(distancia_total_eval)


    # Opcional: Imprimir la recompensa total del episodio para seguir el progreso
    if (episodio + 1) % 100 == 0:
        print(f"Episodio {episodio + 1}/{num_episodios}, Recompensa total: {recompensa_total_episodio:.2f}, Distancia Evaluada: {distancia_total_eval:.2f}")

print("Entrenamiento completado.")

# --- Graficando la convergencia ---
plt.figure(figsize=(30, 10))
plt.plot(range(1, num_episodios + 1), episode_distances)
plt.xlabel("Episodio")
plt.ylabel("Distancia Total del Camino Encontrado (Evaluación)")
plt.title("Convergencia del Agente Q-Learning")
plt.grid(True)
plt.show()

# --- Encuentra el mejor camino después del entrenamiento (usando la tabla Q final) ---

print("\nBuscando el mejor camino encontrado por el agente después del entrenamiento:")

# Comienza la evaluación desde la ciudad que fue el inicio del último episodio de entrenamiento
ciudad_inicio_final_eval = ciudad_inicio
camino_encontrado_final = [ciudad_inicio_final_eval]
ciudad_actual_final_eval = ciudad_inicio_final_eval
bitmask_visitadas_final_eval = 1 << ciudad_inicio_final_eval
distancia_total_final_eval = 0

while bitmask_visitadas_final_eval < (2**num_ciudades) - 1:
    estado_index_final_eval = get_estado_index(ciudad_actual_final_eval, bitmask_visitadas_final_eval)
    acciones_posibles_final_eval = get_acciones_posibles(bitmask_visitadas_final_eval)

    if not acciones_posibles_final_eval:
        print("Advertencia: El camino de evaluación no visitó todas las ciudades.")
        distancia_total_final_eval = float('inf')
        break

    q_valores_actuales_final_eval = q_table[estado_index_final_eval]
    q_valores_posibles_final_eval = [q_valores_actuales_final_eval[accion] for accion in acciones_posibles_final_eval]

    if not q_valores_posibles_final_eval:
        distancia_total_final_eval = float('inf')
        break
    else:
        max_q_final_eval = max(q_valores_posibles_final_eval)
        # En la evaluación final, siempre elige la acción con el valor Q más alto (incluso si es negativo, para seguir la política aprendida)
        mejores_acciones_final_eval = [acciones_posibles_final_eval[i] for i, q_val in enumerate(q_valores_posibles_final_eval) if q_val == max_q_final_eval]
        # Elige la primera mejor acción para una evaluación determinista
        proxima_ciudad_final_eval = mejores_acciones_final_eval[0]


    distancia_total_final_eval += matriz_distancias[ciudad_actual_final_eval, proxima_ciudad_final_eval]
    camino_encontrado_final.append(proxima_ciudad_final_eval)

    ciudad_actual_final_eval = proxima_ciudad_final_eval
    bitmask_visitadas_final_eval = actualizar_bitmask(bitmask_visitadas_final_eval, proxima_ciudad_final_eval)

# Agrega la distancia para regresar al inicio si se completó un tour completo
if bitmask_visitadas_final_eval == (2**num_ciudades) - 1 and ciudad_actual_final_eval != ciudad_inicio_final_eval:
    distancia_total_final_eval += matriz_distancias[ciudad_actual_final_eval, ciudad_inicio_final_eval]
    camino_encontrado_final.append(ciudad_inicio_final_eval)
elif bitmask_visitadas_final_eval < (2**num_ciudades) - 1:
    # Si el camino fue incompleto
     print("Advertencia: El camino de evaluación final no visitó todas las ciudades.")
     distancia_total_final_eval = float('inf')


print("Camino encontrado:", " -> ".join(map(str, camino_encontrado_final)))
print(f"Distancia total del camino encontrado: {distancia_total_final_eval:.2f}")


# --- Graficando el camino final con distancias ---
if coords_input.lower() == 'a' or coords_input.lower() == 'm':

    plt.figure(figsize=(8, 6))
    # Graficar las ciudades
    x_coords = [c[0] for c in coordenadas]
    y_coords = [c[1] for c in coordenadas]
    plt.scatter(x_coords, y_coords, marker='o', s=100, color='red')
    # Etiquetar las ciudades
    for i in range(num_ciudades):
        plt.text(x_coords[i], y_coords[i], str(i), fontsize=12, ha='right')

    # Graficar el camino encontrado y añadir distancias
    if len(camino_encontrado_final) > 1 and distancia_total_final_eval != float('inf'):
        camino_x_final = [coordenadas[ciudad][0] for ciudad in camino_encontrado_final]
        camino_y_final = [coordenadas[ciudad][1] for ciudad in camino_encontrado_final]
        plt.plot(camino_x_final, camino_y_final, linestyle='-', color='blue', marker='o')

        # Añade etiquetas de distancia al gráfico para los segmentos DENTRO del tour
        for i in range(len(camino_encontrado_final) - 1):
            ciudad1_index = camino_encontrado_final[i]
            ciudad2_index = camino_encontrado_final[i+1]
            if ciudad1_index != ciudad2_index: # Evita etiquetar la distancia de una ciudad a sí misma
                 distance = matriz_distancias[ciudad1_index, ciudad2_index]
                 mid_x = (coordenadas[ciudad1_index][0] + coordenadas[ciudad2_index][0]) / 2
                 mid_y = (coordenadas[ciudad1_index][1] + coordenadas[ciudad2_index][1]) / 2
                 plt.text(mid_x, mid_y, f"{distance:.2f}", fontsize=9, color='black', ha='center', va='bottom')


        # Grafica la línea de regreso a la ciudad de inicio para cerrar el ciclo
        if len(camino_encontrado_final) > num_ciudades and camino_encontrado_final[0] == camino_encontrado_final[-1]:
             ciudad_final_index = camino_encontrado_final[-2]
             ciudad_inicio_index = camino_encontrado_final[0]
             if ciudad_final_index != ciudad_inicio_index:
                 distance_return = matriz_distancias[ciudad_final_index, ciudad_inicio_index]
                 mid_x_return = (coordenadas[ciudad_final_index][0] + coordenadas[ciudad_inicio_index][0]) / 2
                 mid_y_return = (coordenadas[ciudad_final_index][1] + coordenadas[ciudad_inicio_index][1]) / 2
                 plt.plot([coordenadas[ciudad_final_index][0], coordenadas[ciudad_inicio_index][0]],
                          [coordenadas[ciudad_final_index][1], coordenadas[ciudad_inicio_index][1]],
                          linestyle='--', color='gray')
                 plt.text(mid_x_return, mid_y_return, f"{distance_return:.2f}", fontsize=9, color='black', ha='center', va='bottom')


    plt.title("Camino final encontrado por el Agente Q-Learning con Distancias")
    plt.xlabel("Coordenada X")
    plt.ylabel("Coordenada Y")
    plt.grid(True)
    plt.show()